In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint

/home/woomurf/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sess = tf.InteractiveSession()

In [3]:
input_dim = 5   # one-hot 한 갯수, 즉 h안의 수가 5개 이므로 5임. 
sequence_length = 6  # 우리가 넣을 문자열의 길이 ex) hihell == 6
hidden_size = 5 # 결과값도 one-hot으로 나와야 하기 때문에 같은 5
batch_size = 1
num_classes = 5

In [4]:
idx2char = ['h','i','e','l','o']
x_data = [[0,1,0,2,3,3]] # hihell
x_one_hot = [[
    [1,0,0,0,0], # h
    [0,1,0,0,0], # i
    [1,0,0,0,0], # h
    [0,0,1,0,0], # e
    [0,0,0,1,0], # l
    [0,0,0,1,0] # l
]]

y_data = [[1,0,2,3,3,4]] # ihello -> hihello

X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])  # [None, 6, 5]
Y = tf.placeholder(tf.int32, [None,sequence_length])  # [None, 6]

## Sequence_loss 사용 예시

In [ ]:
cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype = tf.float32)

In [ ]:
y_data = tf.constant([[1,1,1]])

prediction = tf.constant([[[0.2,0.7], [0.6,0.2], [0.2,0.9]]], dtype = tf.float32)

weights = tf.constant([[1,1,1]], dtype = tf.float32)

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = prediction, targets = y_data, weights=weights)
sess.run(tf.global_variables_initializer())
print("Loss : ", sequence_loss.eval())

# sequence_loss를 이용해 loss를 구할 수 있다.

## 예측 및 훈련해보기

In [6]:
cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype = tf.float32)

# X_for_fc = tf.reshape(outputs, [-1, hidden_size])
# outputs = tf.contrib.layers.fully_connected(
#     inputs = X_for_fc, num_outputs = num_classes, activation_fn = None
# )

# outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits = outputs, targets = Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate = 0.1).minimize(loss)

# rnn에서 나오는 outputs을 logit에 넣으면 좋지 않다 원래는
# 지금은 간단하게 하기 위해서 넣는 것

In [8]:
prediction = tf.argmax(outputs, axis = 2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(2000):
        l, _ = sess.run([loss,train], feed_dict={X:x_one_hot, Y:y_data})
        result = sess.run(prediction, feed_dict={X:x_one_hot})
        print(i,"loss: ", l, "prediction: ", result, "true Y: ", y_data)
        
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\t Prediction str: ", ''.join(result_str))

0 loss:  1.6529268 prediction:  [[4 2 2 2 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  oeeell
1 loss:  1.5627731 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  eeelll
2 loss:  1.488521 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  eeelll
3 loss:  1.4073691 prediction:  [[2 2 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  eelllo
4 loss:  1.3350195 prediction:  [[2 2 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  eelllo
5 loss:  1.2790983 prediction:  [[2 2 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  eelllo
6 loss:  1.2348676 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  eeelll
7 loss:  1.2025502 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  eeelll
8 loss:  1.1818397 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  eeelll
9 loss:  1.1631291 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 

98 loss:  0.65883017 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
99 loss:  0.65855956 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
100 loss:  0.6583026 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
101 loss:  0.6580576 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
102 loss:  0.65782213 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
103 loss:  0.6575951 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
104 loss:  0.6573757 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
105 loss:  0.65716344 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
106 loss:  0.6569581 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
107 loss:  0.6567594 prediction:  [[1 0 2 3 

184 loss:  0.6491605 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
185 loss:  0.649109 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
186 loss:  0.64904755 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
187 loss:  0.648995 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
188 loss:  0.6489386 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
189 loss:  0.6488824 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
190 loss:  0.64883065 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
191 loss:  0.64877343 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
192 loss:  0.6487223 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
193 loss:  0.64866805 prediction:  [[1 0 2 3 

277 loss:  0.6453428 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
278 loss:  0.6453129 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
279 loss:  0.64528316 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
280 loss:  0.64525354 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
281 loss:  0.6452242 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
282 loss:  0.6451948 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
283 loss:  0.64516574 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
284 loss:  0.6451368 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
285 loss:  0.645108 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
286 loss:  0.6450793 prediction:  [[1 0 2 3 

367 loss:  0.6431723 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
368 loss:  0.64315283 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
369 loss:  0.6431335 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
370 loss:  0.6431142 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
371 loss:  0.643095 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
372 loss:  0.6430759 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
373 loss:  0.64305687 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
374 loss:  0.6430379 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
375 loss:  0.64301896 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
376 loss:  0.64300025 prediction:  [[1 0 2 3

...

1547 loss:  0.63572764 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1548 loss:  0.6357252 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1549 loss:  0.63572276 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1550 loss:  0.63572025 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1551 loss:  0.63571775 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1552 loss:  0.6357152 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1553 loss:  0.63571274 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1554 loss:  0.63571024 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1555 loss:  0.63570774 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1556 loss:  0.63570523 predict

1650 loss:  0.6354798 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1651 loss:  0.6354775 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1652 loss:  0.63547516 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1653 loss:  0.6354729 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1654 loss:  0.6354705 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1655 loss:  0.63546824 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1656 loss:  0.63546586 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1657 loss:  0.63546365 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1658 loss:  0.63546133 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1659 loss:  0.63545907 predictio

1750 loss:  0.63525647 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1751 loss:  0.6352543 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1752 loss:  0.6352522 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1753 loss:  0.63525003 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1754 loss:  0.6352479 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1755 loss:  0.6352457 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1756 loss:  0.63524354 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1757 loss:  0.6352414 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1758 loss:  0.6352393 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1759 loss:  0.63523716 prediction:

1836 loss:  0.6350763 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1837 loss:  0.63507426 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1838 loss:  0.63507223 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1839 loss:  0.6350702 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1840 loss:  0.6350682 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1841 loss:  0.63506615 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1842 loss:  0.6350641 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1843 loss:  0.6350621 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1844 loss:  0.6350601 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1845 loss:  0.635058 prediction:  

1925 loss:  0.63489985 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1926 loss:  0.634898 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1927 loss:  0.63489616 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1928 loss:  0.63489413 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1929 loss:  0.6348922 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1930 loss:  0.6348903 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1931 loss:  0.6348884 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1932 loss:  0.63488644 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1933 loss:  0.63488454 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	 Prediction str:  ihello
1934 loss:  0.6348826 prediction: